# Prepare dataset

In [1]:
import os,glob
import open3d
from tqdm import tqdm
import numpy as np
import trimesh
from easydict import EasyDict
import pandas as pd

In [2]:
label_dict={
    0:'botella',
    1:'cantaro',
    2: 'cuenco',
    3: 'figurina',
    4:'lebrillo',
    5:'olla',
    6:'plato',
    7:'vaso'
}

In [3]:
def as_mesh(scene_or_mesh):
    if isinstance(scene_or_mesh, trimesh.Scene):
        mesh = trimesh.util.concatenate([
            trimesh.Trimesh(vertices=m.vertices, faces=m.faces)
            for m in scene_or_mesh.geometry.values()])
    else:
        mesh = scene_or_mesh
    return mesh

In [4]:
def prepare_edges(mesh):
    vertices = mesh['vertices']
    faces = mesh['faces']
    mesh['edges'] = [set() for _ in range(vertices.shape[0])]
    for i in range(faces.shape[0]):
        for v in faces[i]:
            mesh['edges'][v] |= set(faces[i])
    for i in range(vertices.shape[0]):
        if i in mesh['edges'][i]:
            mesh['edges'][i].remove(i)
        mesh['edges'][i] = list(mesh['edges'][i])
    max_vertex_degree = np.max([len(e) for e in mesh['edges']])
    for i in range(vertices.shape[0]):
        if len(mesh['edges'][i]) < max_vertex_degree:
            mesh['edges'][i] += [-1] * (max_vertex_degree - len(mesh['edges'][i]))
    mesh['edges'] = np.array(mesh['edges'], dtype=np.int32)


In [5]:
def add_fields_and_dump_model(mesh_data, fileds_needed, out_fn, dump_model=True):
    m = {}
    for k, v in mesh_data.items():
        if k in fileds_needed:
            m[k] = v
    for field in fileds_needed:
        if field not in m.keys():
            if field == 'edges':
                prepare_edges(m)

    if dump_model:
        np.savez(out_fn, **m)
    return m

In [6]:
def remesh(mesh_orig, target_n_faces):
    if target_n_faces < np.asarray(mesh_orig.triangles).shape[0]:
        mesh = mesh_orig.simplify_quadric_decimation(target_n_faces)
        str_to_add = '_simplified_to_' + str(target_n_faces)
        mesh = mesh.remove_unreferenced_vertices()
    else:
        mesh = mesh_orig
        str_to_add = '_not_changed_' + str(np.asarray(mesh_orig.triangles).shape[0])
    return mesh, str_to_add

In [7]:
def load_mesh(model_fn, classification=True):
  # To load and clean up mesh - "remove vertices that share position"
    if classification:
        mesh_ = trimesh.load_mesh(model_fn, process=True)
        mesh_ = as_mesh(mesh_)
        mesh_.remove_duplicate_faces()
    else:
        mesh_ = trimesh.load_mesh(model_fn, process=False)
    mesh = open3d.geometry.TriangleMesh()
    mesh.vertices = open3d.utility.Vector3dVector(mesh_.vertices)
    mesh.triangles = open3d.utility.Vector3iVector(mesh_.faces)
    return mesh

In [8]:
def prepare_directory(filenames, p_out=None, n_target_faces=[4000], verbose=True, classification=True):
    fileds_needed = ['vertices', 'faces', 'edges','label']
    for i in range (len(label_dict)):
        out_path = os.path.join(p_out,label_dict[i])
        if not os.path.isdir(out_path):
            os.makedirs(out_path)

    for file,label in tqdm(filenames, disable=1 - verbose):
        out_fn = os.path.join(p_out,label_dict[label],os.path.split(file)[1].split('.')[0])
        mesh_orig = load_mesh(file, classification=classification)
        for this_target_n_faces in n_target_faces:
            mesh, str_to_add = remesh(mesh_orig, this_target_n_faces)
            mesh_data = EasyDict({'vertices': np.asarray(mesh.vertices), 'faces': np.asarray(mesh.triangles), 'label': label})
            out_fc_full = out_fn + str_to_add
            add_fields_and_dump_model(mesh_data, fileds_needed, out_fc_full)

In [9]:
def prepare_datasetShape(csv_file,dataset_folder_path,pout):
    n_target_faces = [4000]
    label = label_dict
    pin = '/home/raito/Downloads/datasetShape/train'
    df = pd.read_csv(csv_file)
    file = df['obj_id'].apply(lambda x: os.path.join(dataset_folder_path,str(x)+'.obj'))
    filenames= zip(file,df['class_id'])
    
    prepare_directory(filenames,p_out=pout,n_target_faces=n_target_faces)

In [10]:
file_csv = '/home/raito/Downloads/1_train.csv'
pout = './train/'
dataset = '/home/raito/Downloads/datasetShape/train'
prepare_datasetShape(file_csv,dataset,pout)

528it [05:19,  1.65it/s]


In [11]:
file_csv = '/home/raito/Downloads/1_val.csv'
pout = './val/'
dataset = '/home/raito/Downloads/datasetShape/train'
prepare_datasetShape(file_csv,dataset,pout)

133it [01:20,  1.65it/s]


# Dataset

In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder,DatasetFolder
import walks
import params_setting

In [2]:
def load_model_from_npz(npz_fn):
    if npz_fn.find(':') != -1:
        npz_fn = npz_fn.split(':')[1]
    mesh_data = np.load(npz_fn, encoding='latin1', allow_pickle=True)
    return mesh_data

In [14]:
test = load_model_from_npz('/home/raito/Downloads/pytorch-meshwalker/train/botella/89_simplified_to_4000.npz')

In [15]:
print(test['label'])

0


In [3]:
def fill_xyz_features(features, f_idx, vertices, mesh_extra, seq, jumps, seq_len):
    walk = vertices[seq[1:seq_len + 1]]
    features[:, f_idx:f_idx + walk.shape[1]] = walk
    f_idx += 3
    return f_idx


def fill_dxdydz_features(features, f_idx, vertices, mesh_extra, seq, jumps, seq_len):
    walk = np.diff(vertices[seq[:seq_len + 1]], axis=0) * 100
    features[:, f_idx:f_idx + walk.shape[1]] = walk
    f_idx += 3
    return f_idx

In [4]:
def generate_walk(mesh_data, labels_from_npz,dataset_params):
#     mesh_data = {'vertices': vertices.numpy(),
#                'faces': faces.numpy(),
#                'edges': edges.numpy(),
#                }
#     if dataset_params_list[params_idx[0]].label_per_step:
#         mesh_data['labels'] = labels_from_npz.numpy()

#     dataset_params = dataset_params_list[params_idx[0].numpy()]
    features = mesh_data_to_walk_features(mesh_data, dataset_params)

#     if dataset_params_list[params_idx[0]].label_per_step:
#         labels_return = labels
#     else:
    labels_return = labels_from_npz

    return features, labels_return

In [5]:
def norm_model(vertices):
  # Move the model so the bbox center will be at (0, 0, 0)
    mean = np.mean((np.min(vertices, axis=0), np.max(vertices, axis=0)), axis=0)
    vertices -= mean

  # Scale model to fit into the unit ball
    norm_with = np.max(vertices)
    vertices /= norm_with

#     if norm_model.sub_mean_for_data_augmentation:     # TODO: check again
#         vertices -= np.nanmean(vertices, axis=0)

In [6]:
def mesh_data_to_walk_features(mesh_data, dataset_params):
    vertices = mesh_data['vertices']
    seq_len = dataset_params.seq_len
    if dataset_params.set_seq_len_by_n_faces:
        seq_len = int(mesh_data['vertices'].shape[0])
        seq_len = min(seq_len, dataset_params.seq_len)

  # Preprocessing
    if dataset_params.adjust_vertical_model:
        vertices[:, 1] -= vertices[:, 1].min()
    if dataset_params.normalize_model:
        norm_model(vertices)

  # Data augmentation
#     for data_augmentaion_function in dataset_params.data_augmentaion_vertices_functions:
#         data_augmentaion_function(vertices)

  # Get essential data from file
#     if dataset_params.label_per_step:
#         mesh_labels = mesh_data['labels']
#     else:
#         mesh_labels = -1 * np.ones((vertices.shape[0],))

    mesh_extra = {}
    mesh_extra['n_vertices'] = vertices.shape[0]
    if dataset_params.edges_needed:
        mesh_extra['edges'] = mesh_data['edges']

    features = np.zeros((dataset_params.n_walks_per_model, seq_len, dataset_params.number_of_features), dtype=np.float32)
#     labels   = np.zeros((dataset_params.n_walks_per_model, seq_len), dtype=np.int32)

    for walk_id in range(dataset_params.n_walks_per_model):
        f0 = np.random.randint(vertices.shape[0])                               # Get walk starting point
        seq, jumps = dataset_params.walk_function(mesh_extra, f0, seq_len)      # Get walk indices (and jump indications)

        f_idx = 0
        for fill_ftr_fun in dataset_params.fill_features_functions:
            f_idx = fill_ftr_fun(features[walk_id], f_idx, vertices, mesh_extra, seq, jumps, seq_len)
#         if dataset_params.label_per_step:
#             labels[walk_id] = mesh_labels[seq[1:seq_len + 1]]

    return features

In [7]:
def setup_features_params(dataset_params):
    if dataset_params.uniform_starting_point:
        dataset_params.area = 'all'
    else:
        dataset_params.area = -1
    norm_model.sub_mean_for_data_augmentation = dataset_params.sub_mean_for_data_augmentation
    dataset_params.support_mesh_cnn_ftrs = False
    dataset_params.fill_features_functions = []
    dataset_params.number_of_features = 0
    net_input = dataset_params.net_input
    if 'xyz' in net_input:
        dataset_params.fill_features_functions.append(fill_xyz_features)
        dataset_params.number_of_features += 3
    if 'dxdydz' in net_input:
        dataset_params.fill_features_functions.append(fill_dxdydz_features)
        dataset_params.number_of_features += 3
#     if 'vertex_indices' in net_input:
#         dataset_params.fill_features_functions.append(fill_vertex_indices)
#         dataset_params.number_of_features += 1

    dataset_params.edges_needed = True
    if dataset_params.walk_alg == 'random_global_jumps':
        dataset_params.walk_function = walks.get_seq_random_walk_random_global_jumps
    else:
        raise Exception('Walk alg not recognized: ' + params.walk_alg)
    return dataset_params.number_of_features

In [8]:
class ShapeDataset(DatasetFolder):
    def __init__(self, root,seq_length,param,transform=None):
        super().__init__(root,loader = None,extensions=('.npz'))
        self.transform = transform
        self.param =param
        self.param.label_per_step = False
        self.param.seq_len=seq_length
        self.param.set_seq_len_by_n_faces=False
        self.number_features = setup_features_params(self.param)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        mesh_data = load_model_from_npz(path)
        features, label = generate_walk(mesh_data,mesh_data['label'],self.param)
        if self.transform:
            features = self.transform(features)
        return features, label

# General model 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [79]:
class RnnWalkNet(nn.Module):
    def __init__(self, num_class=8,batch_size=4,dataset_params = None):
        super().__init__()
        self.batch_size = batch_size
        self.num_class = num_class
        self.param = dataset_params
        self.input_dim = dataset_params.number_of_features
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=self.input_dim ,out_features=128),
            nn.BatchNorm1d(100),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=128 ,out_features=256),
            nn.BatchNorm1d(100),
            nn.ReLU()
        )
      
        self.gru1=nn.GRU(256, 1024, batch_first=True,dropout=0)
        self.gru2=nn.GRU(1024, 1024, batch_first=True,dropout=0)
        self.gru3=nn.GRU(1024, 512, batch_first=True,dropout=0)
        self.fc3 =nn.Linear(in_features=512, out_features=self.num_class)

    def forward(self, x):
        batch_first = True
        x = torch.squeeze(x)
        out = self.fc1(x)
        out = self.fc2(out)
        self.h_01 = torch.empty(1,out.shape[0], 1024).cuda()
        self.h_02 = torch.empty(1,out.shape[0], 1024).cuda()
        self.h_03 = torch.empty(1,out.shape[0], 512).cuda()
        nn.init.orthogonal_(self.h_01)
        nn.init.orthogonal_(self.h_02)
        nn.init.orthogonal_(self.h_03)
        out1,h_1 = self.gru1(out,self.h_01)
        out2,h_2 = self.gru2(out1,self.h_02)
        out3,h_3 = self.gru3(out2,self.h_03)
        #out3 shape = [batch,seq_len_feature] => need [batch,features] of last hidden state
        out = torch.squeeze(h_3)
        out =self.fc3(out)
        return out


# Pytorch lightning model

In [11]:
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from pytorch_lightning.metrics.functional import accuracy
from torch import nn
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
from torchvision import models, transforms

In [76]:
class MeshWalkerShrecTrack1(pl.LightningModule):
    """
    Args:
        num_class: number of output classes
    """
    def __init__(self, model, class_weight=None):
        super().__init__()
        self.model = model
        self.class_weights = class_weight

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: input image, shape (B, C, H, W)
        Returns:
            torch.Tensor: output logits tensor, shape (B, num_class)
        """
        out: torch.Tensor = self.model(x)
        return out

    def training_step(self, batch, batch_idx):
        """
        """
        x, y = batch
        out = self.model(x)

        weight = torch.tensor(self.class_weights).to(
            x.device) if self.class_weights else None

        loss = F.cross_entropy(out, y, weight=weight)

        self.log('train_loss', loss, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """
        """
        x, y = batch
        logits = self.model(x)

        weight = torch.tensor(self.class_weights).to(
            x.device) if self.class_weights else None

        loss = F.cross_entropy(logits, y, weight=weight)

        preds = torch.argmax(F.log_softmax(logits, dim=1), dim=1)

        self.log('val_loss', loss)

        return {'val_loss': loss, 'labels': y, 'preds': preds}

    def validation_epoch_end(self, outputs):
        """
        """
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        labels = torch.cat([x['labels'] for x in outputs])
        preds = torch.cat([x['preds'] for x in outputs])

        acc = accuracy(preds, labels)

        tensorboard_log = {'val_loss': avg_loss}

        self.log('val_acc', acc, prog_bar=True, logger=True)
        self.log('val_loss', avg_loss, prog_bar=True, logger=True)


    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.model(x)
        preds = torch.argmax(F.log_softmax(logits, dim=1), dim=1)

        return {'labels': y, 'preds': preds}

    def test_epoch_end(self, outputs):
        labels = torch.cat([x['labels'] for x in outputs])
        preds = torch.cat([x['preds'] for x in outputs])

        acc = accuracy(preds, labels)
#         xray_metrics = covid_xray_metrics(labels, preds)

        return {
            'test_acc': acc,
#             'sensitivity': xray_metrics['sensitivity'],
#             'ppv': xray_metrics['ppv'],
#             'cm': xray_metrics['cm']
        }

    def configure_optimizers(self):
        """
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=5e-4)
#         lr_dict = {
#             'scheduler': ReduceLROnPlateau(optimizer, patience=5),
#             'interval': 'epoch', # The unit of the scheduler's step size
#             'frequency': 1, # The frequency of the scheduler
#             'reduce_on_plateau': True, # For ReduceLROnPlateau scheduler
#             'monitor': 'val_loss', # Metric for ReduceLROnPlateau to monitor
#             'strict': True, # Whether to crash the training if `monitor` is not found
#         }

#         return [optimizer], [lr_dict]
        return optimizer

In [13]:
from argparse import ArgumentParser
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
param =  params_setting.set_up_default_params('classification','track1')
SEED=1902
pl.seed_everything(SEED)
LOSS_WEIGHT=None
  
train_set = ShapeDataset('./train/',100,param)
valid_set = ShapeDataset('./val/',100,param)

    # train_loader_folder = create_balance_dl(train_set,
    #                                         batch_size=16,
    #                                         num_workers=12)

train_loader_folder = DataLoader(train_set,
                                     batch_size=16,
                                     shuffle=True,
                                     num_workers=12,
                                     pin_memory=True)

valid_loader_folder = DataLoader(valid_set,
                                     batch_size=16,
                                     shuffle=False,
                                     num_workers=12,
                                     pin_memory=True)

backbone = RnnWalkNet(num_class=8,batch_size=16,dataset_params=param)
model = MeshWalkerShrecTrack1(backbone, class_weight=LOSS_WEIGHT)
checkpoint_path = './lightning_logs/version_18/checkpoints/epoch=335-step=11063.ckpt'
checkpoint_callback = ModelCheckpoint(monitor='val_loss',mode='min', save_top_k=5)
# trainer = pl.Trainer(gpus=1,max_epochs=3000,checkpoint_callback=checkpoint_callback)
trainer = pl.Trainer(gpus=1,max_epochs=3000,checkpoint_callback=checkpoint_callback,resume_from_checkpoint=checkpoint_path)
trainer.fit(model, train_loader_folder, valid_loader_folder)


Global seed set to 1902
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | RnnWalkNet | 12.6 M
-------------------------------------
12.6 M    Trainable params
0         Non-trainable params
12.6 M    Total params
Restored states from the checkpoint file at ./lightning_logs/version_18/checkpoints/epoch=335-step=11063.ckpt


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]